In [1]:
#hide
import sys
from IPython.display import HTML, display, Image, YouTubeVideo
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import proplot as plot

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import (
    plotting as myplt,
    animation as myanim,
    coupling as bl,
    utils,
    matrix_lattice as ml,
    envelope_analysis as ea
)
from tools.utils import play, show
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['animation.embed_limit'] = np.inf

# Parametric Oscillators
> Studying particle motion in an ideal model of an accelerator.

- toc: true 
- badges: false
- comments: true
- categories: [physics, accelerators]
- image: images/chart-preview.png

This post presents the solution to a general problem: what is the motion of a particle in one dimension (1D) in the presence of time-dependent linear forces? This amounts to solving the following equation of motion:

$$\frac{d^2x}{dt^2} + k(t)x = 0,$$

where $k(t + T) = k(t)$ for some $T$. This is a [parametric oscillator](https://en.wikipedia.org/wiki/Parametric_oscillator), a harmonic oscillator whose physical properties are not static. For example, the oscillations of a pendulum (in the small angle approximation) on the surface of a planet whose gravitational pull varies periodically would be described by the above equation. The solution to this equation was derived by [George William Hill](https://en.wikipedia.org/wiki/George_William_Hill) in 1886 to study lunar motion, and for this reason it is known as *Hill's equation*. It also finds application in areas such as condensed matter physics, quantum optics, and accelerator physics. After setting up the physical problem, we will examine the solutions and discuss their relevance to the last application, accelerator physics.

## Problem motivation 

### Particle accelerator physics 

[Particle accelerators](https://en.wikipedia.org/wiki/Particle_accelerator) are machines which produce groups of charged particles (known as *beams*), increase their kinetic energy, and guide them to a target. These machines are invaluable to modern scientific research. The most famous examples are *colliders*, such as the [LHC](https://home.cern/science/accelerators/large-hadron-collider), in which two beams are smashed together to generate fundamental particles. A lesser known fact is that the fields of condensed matter physics, material science, chemistry, and biology also benefit tremendously from accelerators; this is due to the effectiveness of scattering experiments in which the deflection of a beam after colliding with a target is used to learn information about the target. The scattered beam is composed of neutrons in *spallation neutron sources* such as [SNS](https://neutrons.ornl.gov/sns), electrons in *electron scattering facilities* such as [CEBAF](https://science.osti.gov/np/Facilities/User-Facilities/CEBAF), or photons in *synchrotron light sources* such as [APS](https://www.aps.anl.gov). In addition to scientific research, accelerators find use in [medicine](http://www.accelerators-for-society.org/health/index.php?id=7), particularly for cancer treatment, and also in various [industrial applications](http://www.accelerators-for-society.org/industry/index.php?id=8).

<img src="images/lhc.jpg" width="500">
<center>A large detector at an interaction point in the LHC.</center>

There are generally a few beam properties which are very important to experimentalists; in colliders it is the energy and luminosity, in spallation sources it is the intensity, and in light sources it is the brightness. There is thus a constant need to push these parameters to new regions. For example, below is the famous Livingston plot which shows the energy achieved by various machines over the past century (note on energy scale).

<img src="images/livingston.png" width="400">

There are many physics issues associated with the optimization of these beam parameters. *Accelerator physics* is a field of applied physics which studies these issues. The task of the accelerator physicist is to understand, control, and measure the journey of the beam from its creation to its final destination. The difficulty of this task has grown over time; the improvement accelerator performance has brought with it a staggering increase in size and complexity. The construction and operation of modern accelerators generally requires years of planning, thousands of scientists and engineers, and hundreds of millions or even billions of dollars. We next discuss the physics principles upon which these machines are built, which are quite simple despite this complexity.

### Linear model of an accelerator

There are three basic tasks an accelerator has to accomplish. First, it must increase the beam energy (acceleration). Second, it must guide the beam along a predetermined path (steering). Third, it must ensure the beam particles remain close together (focusing). How can we accomplish this? Well, particles are charged, and the force on a point charge in electromagnetic fields is given by

$$\mathbf{F} = q\left({\mathbf{E} + \mathbf{v} \times \mathbf{B}}\right),$$

where $q$ is the particle charge, $\mathbf{v}$ is the particle velocity, $\mathbf{E}$ is the electric field, and $\mathbf{B}$ is the magnetic field. An accelerator consists of a series of elements, each with their own $\mathbf{E}$ and $\mathbf{B}$. The collection of these elements is called a *lattice*. Before discussing how these fields are created, we should define a coordinate system. It is conventional to define an $s$ axis pointing pointing along the design trajectory, and the $x$ and $y$ axes defined in the plane transverse to $s$. In this way the motion is broken up into transverse ($x$ and $y$) and longitudinal ($s$) dynamics. It is also conventional to use the transverse slope $x' = dx/ds$ instead of the velocity.

<img src="images/coordinate_system.png">

The first task, acceleration, is not the focus of this post; I'll just mention the basic principles that are used. A simple method is to produce a voltage difference between two conductors, but there is a limit to the electric field strengths that can be produced in this way. The solution is to create a series of radio-frequency (RF) cavities, each with its own oscillating $\mathbf{E}$ field. The positions and lengths of these cavities are chosen so that particles are only within the cavities when the electric field is pointed along the direction of motion, as shown by this fantastic animation from Wikipedia:

<img src="images/linac.gif" width="600">

The remaining tasks, steering and focusing, concern the motion in the transverse plane. Magnetic fields, not electric fields, are used since their effect grows with increased particle energy. The magnetic field used for steering is the dipole field given by 

$$\mathbf{B}_{dipole} \propto \hat{y}.$$

This will simply bend the particle in a circle in the horizontal plane. A dipole field can be formed by pointing the north pole of a magnet at the south pole of a different magnet. Next, quadrupole fields are used for focusing:

$$\mathbf{B}_{quad} \propto y\hat{x} + x\hat{y}.$$

A quadrupole field can be obtained by overlaying two dipole fields and rotating them 90 degrees relative to each other. These fields are shown below.

<img src="images/dipole_quadrupole.png">

The force from the quadrupole is focusing in the horizontal direction, but *defocusing* in the vertical direction; however, net focusing is still achieved by alternating the direction of the quadrupoles. This is analogous to a beam of light passign through a series of converging and diverging lenses. If the spacing and curvature of the lenses is correctly chosen, a net focusing can be achieved. In this sense a quadrupole acts as a  magnetic lense.

<img src="images/thinlens.png">

### Equation of motion

Dipoles and quadrupoles form the core of modern accelerators. For example, the LHC has around 1200 dipoles. This [video](https://www.youtube.com/watch?v=qTEFZEIWUdo) shows a bit of the construction process for these magnets. In reality there will be no such thing as a *pure* dipole or quadrupole field, i.e., there will be higher order moments resulting in nonlinear forces; however, the approximation is usually reasonable. If we make this approximation, the forces from dipoles and quadrupoles are *linear*, meaning they are proportional to the particle position, and also *uncoupled*, meaning the motion in one dimension is independent of the motion in the other dimension. We thus arrive at our original equation of motion:

$$x'' + k(s)x = 0,$$

where $k(s + L) = k(s)$ for some distance $L$. The period length $L$ could be the entire circumference of a circular machine, or could be a smaller repeated subsection.

## Solution

### Envelope function

The general solution to Hill's equation is given by

$$x(s) = \sqrt{\epsilon}\,w(s) \cos\left({\mu(s) + \delta}\right).$$

This introduces an amplitude $w(s) = w(s + L)$ which we call the envelope function, as well as a phase $\mu$, both of which depend on $s$. The constants $\epsilon$ and $\delta$ are determined by the initial conditions. Below is an example in what is known as a FODO (focus-off-defocus-off) lattice, which consists of evenly spaced focusing and defocusing quadrupoles. For now we can think of the lattice as repeating itself forever in the $s$ direction. Each black line on the right represents the trajectory for a different initial position and slope; although the individual trajectories look rather complicated, the envelope function has a very simple form.

<img src="images/fodo.png">

### Phase space

The particle motion becomes much easier to interpret if we observe it in position-momentum space, aka *phase space*. The following animation shows the evolution of the particle phase space coordinates at a single position in the lattice. The position shown is $s = nL/4$, where $n$ is the period number, which corresponds to the midpoint between the focusing and defocusing quadrupoles. 

In [2]:
#hide_input
lattice = ml.fodo(0.5, 0.5, 5.0, start='drift')
x = [1, 0, 0, 0]
X = lattice.track_part(x, nturns=40)

anim = myanim.corner_onepart(
    X, 
    dims=('x','xp'), 
    figsize=(6.5, 5), 
    show_history=True, 
    text_fmt='Period = {}',
    fps=3,
    label_kws=dict(fontsize='large'),
    tick_kws=dict(labelsize='large'),
    grid_kws=dict(alpha=0.11),
    history_kws=dict(alpha=0.2, s=12),
    text_kws=dict(fontsize='x-large'),
    s=20, color='k'
)
anim

We see that the particle jumps along the boundary of an ellipse in phase space. The shape and orientation of the ellipse will change if we look at a different position in the lattice, but its area will be the same. So, the motion is determined by the dimensions and oriention of this ellipse throughout the lattice, as well as the location of the paricle on the ellipse boundary. This motivates the definition of the so-called *Twiss parameters*:

$$\beta = w^2, \quad \alpha = -\frac{1}{2}\beta', \quad \gamma = \frac{1 + \alpha^2}{\beta}.$$

The dimensions of the phase space ellipse are nicely described by these parameters:

<img src="images/csellipse.png" width=700>

The maximum extent of the ellipse is determined by $\beta$ in the $x$ direction and $\gamma$ in the $y$ direction. $\alpha$ is proportional to the slope of the $\beta$ function, and so determines the tilt angle of the ellipse. The position of a particle on the ellipse is given by the phase $\mu$. Finally, the invariant of the motion corresponding to the ellipse area is constructed from the Twiss parameters as

$$\epsilon = \beta {x'}^2 + 2\alpha xx' + \gamma x^2$$

for any $x$ and $x'$. The $\beta$ functions and phase advances in both dimensions are extremely important to measure and control in a real machine. Below shows an example of the $\beta$ functions in the SNS accumulator ring.

<img src="images/beta_function_snsring.png">

### Transfer matrices 

A helpful tool to pair with the parameterization we just introduced is the *transfer matrix*, a matrix which connects the phase space coordinates at two different positions:

$$
\begin{bmatrix} x \\ x' \end{bmatrix}_{s + L} = 
\mathbf{M}
\begin{bmatrix} x \\ x' \end{bmatrix}_{s}
$$

The transfer matrix can be written as $\mathbf{M} = \mathbf{V}\mathbf{P}\mathbf{V}^{-1}$, where

$$
\mathbf{V} = \frac{1}{\sqrt{\beta}}
\begin{bmatrix}
  \beta & 0 \\ -\alpha & 1
\end{bmatrix}
$$
and 
$$
\mathbf{P} = \begin{bmatrix}
  \cos\mu & \sin\mu \\ -\sin\mu & \cos\mu
\end{bmatrix}
$$

The effect of $\mathbf{V}^{-1}$ is to deform the phase space ellipse into a circle while preserving its area. $\mathbf{P}$ is then just a rotation in phase space, and $\mathbf{V}$ then transforms back into a tilted ellipse. This is illustrated below.

<img src="images/norm_coords_2D.png">

$\mathbf{V}$ can be thought of as a time-dependent transformation which removes the variance in the focusing strength, turning the parametric oscillator into a simple harmonic oscillator. 

## Conclusion

There is much more to be said about 1D linear motion in an accelerator, but that is the gist of it. Particles move around the surface of an ellipse in phase space, the area of the ellipse is invariant, and the ellipse is the same at a fixed position in the lattice. The motion is completely determined by three parameters: $\alpha$, $\beta$, and $\mu$. The next post will examine how this treatment can be extended to include coupling between the horizontal and vertical planes.